In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 27.8MB/s 
     |████████████████████████████████| 901kB 36.3MB/s 
     |████████████████████████████████| 3.3MB 34.8MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [3]:
tokenizer = AutoTokenizer.from_pretrained("tartuNLP/EstBERT")
model = AutoModelForMaskedLM.from_pretrained("tartuNLP/EstBERT")

In [4]:
pip install estnltk==1.6.7b0

     |████████████████████████████████| 99.4MB 95kB/s 
     |████████████████████████████████| 747kB 49.2MB/s 
     |████████████████████████████████| 1.5MB 30.8MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [5]:
import torch
from typing import MutableMapping, List
from transformers import BertTokenizer, logging, BertModel

logging.set_verbosity(30)
from estnltk.text import Text
from estnltk.taggers import Tagger
from estnltk.layer.layer import Layer
import numpy as np


class BertTagger(Tagger):
    """Tags BERT embeddings."""

    def __init__(self, bert_location: str, sentences_layer: str = 'sentences',
                 token_level: bool = False,
                 output_layer: str = 'bert_embeddings', bert_layers: List[int] = None, method='concatenate'):

        if bert_layers is None:
            bert_layers = [-4, -3, -2, -1]
        else:
            for layer in bert_layers:
                if abs(layer) > 12:
                    msg = "BERT base model only has 12 layers of transformer encoder, chose layers from (-12..-1). It " \
                          "is reasonable to choose layers from the last layers, for example [-4, -3, -2, -1]: last 4 " \
                          "layers. "
                    raise Exception(msg)
        self.conf_param = ('bert_location', 'bert_model', 'tokenizer', 'method', 'token_level', 'bert_layers')
        if bert_location is None:
            msg = "Directory containing BERT model must be specified."
            raise Exception(msg)
        else:
            self.bert_location = bert_location
        if method not in ('concatenate', 'add', 'all'):
            msg = "Method can be 'concatenate', 'add' or 'all'."
            raise Exception(msg)
        self.method = method
        self.output_layer = output_layer
        self.input_layers = [sentences_layer]

        self.bert_model = BertModel.from_pretrained(bert_location, output_hidden_states=True)

        self.tokenizer = BertTokenizer.from_pretrained(self.bert_location)

        self.output_attributes = ['token', 'bert_embedding']

        self.token_level = token_level
        self.bert_layers = bert_layers

    def _make_layer(self, text: Text, layers: MutableMapping[str, Layer], status: dict) -> Layer:
        sentences_layer = layers[self.input_layers[0]]
        embeddings_layer = Layer(name=self.output_layer, text_object=text, attributes=self.output_attributes,
                                 ambiguous=True)

        start, i = 0, 0
        word_spans = []

        for k, sentence in enumerate(sentences_layer):

            for word in sentence:
                word_spans.append((word.start, word.end, word.text))
            sent_text = sentence.enclosing_text

            embeddings = get_embeddings(sent_text, self.bert_model, self.tokenizer, self.method, self.bert_layers)[
                         1:-1]  # first one in start token, and last one is sep token
            tokens = self.tokenizer.tokenize(sent_text)
            assert len(tokens) == len(embeddings)
            if k != 0:  # move the start manually when next sentence starts
                start = word_spans[i][0]

            if self.token_level:  # annotates tokens
                for j, packed in enumerate(zip(embeddings, tokens)):
                    token_emb, token_init = packed[0], packed[1]

                    if not token_init.startswith("#") and j != 0:  # move to next word
                        if start == word_spans[i][1]:  # BERT's wordpiece tokenizer can tokenize differently
                            i += 1  # next word starts
                            word_span = word_spans[i]
                            start = word_span[0]  # the start id of this word
                    if self.method == 'all':
                        embedding = []
                        for tok_emb in token_emb:
                            emb = []
                            for e in tok_emb:
                                emb.append(float(e))
                            embedding.append(emb)
                    else:
                        embedding = [float(t) for t in token_emb]

                    attributes = {'token': token_init, 'bert_embedding': embedding}
                    token = token_init.strip()
                    embeddings_layer.add_annotation((start, start + len(token.replace('#', ''))), **attributes)
                    start += len(token.replace('#', ''))  # adding token length to the current pointer

                i += 1  # move the pointer manually

            else:  # annotates full words, adding the token level embedding together
                collected_tokens = []
                collected_embeddings = []
                for j, packed in enumerate(zip(embeddings, tokens)):
                    token_emb, token_init = packed[0], packed[1]

                    if i == 0 and j == 0:
                        start = 0
                        collected_embeddings.append(token_emb)
                        collected_tokens.append(token_init)
                        start += len(token_init)

                    if i != 0 and j == 0:
                        collected_tokens = [token_init]
                        collected_embeddings = [token_emb]
                        start = word_spans[i][0]

                        start += len(token_init)

                    if not token_init.startswith("#") and j != 0:  # move to next word
                        if start == word_spans[i][1]:  # BERT's wordpiece tokenizer can tokenize differently

                            if collected_embeddings:
                                if self.method == 'all':
                                    embedding = []
                                    for tok_embs in collected_embeddings:
                                        token_embs = []
                                        for embs in tok_embs:
                                            token_embs_emb = []
                                            for emb in embs:
                                                token_embs_emb.append(float(emb))
                                            token_embs.append(token_embs_emb)
                                        embedding.append(token_embs)
                                else:
                                    embedding = [float(t) for t in np.sum(collected_embeddings, 0)]

                                attributes = {'token': collected_tokens, 'bert_embedding': embedding}
                                embeddings_layer.add_annotation((word_spans[i][0], word_spans[i][1]),
                                                                **attributes)

                            i += 1  # next word starts
                            start = word_spans[i][0]  # the start id of this word
                            collected_embeddings = [token_emb]
                            collected_tokens = [token_init]
                            start += len(token_init)
                        else:
                            start += len(token_init)
                            collected_tokens.append(token_init)
                            collected_embeddings.append(token_emb)
                    elif token_init.startswith("#"):
                        collected_tokens.append(token_init)
                        collected_embeddings.append(token_emb)
                        start += len(token_init.replace("#", ''))

                if collected_tokens:
                    if self.method == 'all':
                        embedding = []
                        for tok_embs in collected_embeddings:
                            token_embs = []
                            for embs in tok_embs:
                                token_embs_emb = []
                                for emb in embs:
                                    token_embs_emb.append(float(emb))
                                token_embs.append(token_embs_emb)
                            embedding.append(token_embs)
                    else:
                        embedding = [float(t) for t in np.sum(collected_embeddings, 0)]

                    attributes = {'token': collected_tokens, 'bert_embedding': embedding}
                    embeddings_layer.add_annotation((word_spans[i][0], word_spans[i][1]),
                                                    **attributes)

                i += 1  # move the pointer manually

        return embeddings_layer


def get_embeddings(sentence: str, model, tokenizer, method, bert_layers):
    input_data = tokenizer.encode_plus(sentence)
    input_ids = input_data.get('input_ids')
    token_vecs_cat = []
    if len(input_ids) > 512:  # maximum sequence length can be 512
        msg = "Input sentence is too big (%s), splitting the sentence." % len(input_ids)
        print(msg)
        collected_input_ids = []
        while True:
            collected_input_ids.append(input_ids[:512])
            input_ids = input_ids[512:]
            if len(input_ids) <= 512:
                collected_input_ids.append(input_ids)
                break
    else:
        collected_input_ids = [input_ids]

    for i, input_ids in enumerate(collected_input_ids):

        segments_ids = [1] * len(input_ids)
        tokens_tensor = torch.tensor([input_ids])
        segments_tensors = torch.tensor([segments_ids])

        with torch.no_grad():
            outputs = model(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]
        token_embeddings = torch.stack(hidden_states, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1, 0, 2)

        for token in token_embeddings:
            if method == 'concatenate':  # concatenate the vectors
                layers = [token[i] for i in bert_layers]
                cat_vec = torch.cat(layers, dim=0)
                token_vecs_cat.append(np.asarray(cat_vec))

            if method == 'add':  # elementwise addition
                layers = [np.asarray(token[i]) for i in bert_layers]
                sum_vec = np.sum(layers, 0)
                token_vecs_cat.append(np.asarray(sum_vec))

            if method == 'all':  # return all
                layers = [np.asarray(token[i]) for i in bert_layers]
                token_vecs_cat.append(layers)

    return token_vecs_cat

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
bert_tagger = BertTagger(bert_location='tartuNLP/EstBERT')

Some weights of the model checkpoint at tartuNLP/EstBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You shoul

In [8]:
import pandas as pd
data = pd.read_csv("Koondtabel_lemmadega.csv")
data#.head()

,Tag,Lõigu sisu,Pealkiri1,Kohtuasja_nr,Kohtuasja_ID,algus_lopp,stop_eemaldatud,Lemma
0,C01PointnumeroteAltN,"27 Kõigepealt väärib märkimist, et väljakujunenud kohtupraktika kohaselt on ELTL artiklis 267 sätestatud liikmesriigi kohtute ja Euroopa Kohtu vahelises koostöömenetluses Euroopa Kohtu ülesanne anda liikmesriigi kohtule tarvilik vastus, mis võimaldaks viimasel poolelioleva kohtuasja lahendada. Seda arvestades tuleb Euroopa Kohtul temale esitatud küsimused vajaduse korral ümber sõnastada (2. aprilli 2020. aasta kohtuotsus Ruska Federacija, C‑897/19 PPU, EU:C:2020:262, punkt 43 ja seal viidatud kohtupraktika).",NaN,C‑416/20 PPU,0,algus,27 väärib märkimist väljakujunenud kohtupraktika eltl artiklis 267 sätestatud liikmesriigi kohtute euroopa kohtu vahelises koostöömenetluses euroopa kohtu ülesanne anda liikmesriigi kohtule tarvilik vastus võimaldaks viimasel poolelioleva kohtuasja lahendada arvestades euroopa kohtul esitatud küsimused vajaduse sõnastada 2. aprilli 2020. aasta kohtuotsus ruska federacija c‑897/19 ppu eu c:2020:262 punkt 43 viidatud kohtupraktika,27 väärima märkimine väljakujunenud kohtupraktika eltl artikkel 267 sätestama liikmesriik kohus euroopa kohus vaheline koostöömenetlus euroopa kohus ülesanne andma liikmesriik kohus tarvilik vastus võimaldama viimane pooleliolev kohtuasi lahendama arvestama euroopa kohus esitama küsimus vajadus sõnastama 2. aprill 2020. aasta kohtuotsus ruskama federacija c ‑ 897 / 19 ppu eu c : 2020 : 262 punkt 43 viidatud kohtupraktika
1,C01PointnumeroteAltN,"28 Käesoleval juhul nähtub eelotsusetaotlusest, et eelotsusetaotluse esitanud kohus peab tegema otsuse selle kohta, kas TRi üleandmine Rumeenia ametiasutustele on õiguspärane seaduse rahvusvahelise õigusabi kohta kriminaalasjades § 83 sätete alusel, millega rakendatakse raamotsuse 2002/584 artiklit 4a Saksa õiguses.",NaN,C‑416/20 PPU,0,algus,28 käesoleval nähtub eelotsusetaotlusest eelotsusetaotluse esitanud kohus otsuse tri üleandmine rumeenia ametiasutustele õiguspärane seaduse rahvusvahelise õigusabi kriminaalasjades § 83 sätete alusel rakendatakse raamotsuse 2002/584 artiklit 4a saksa õiguses,28 käesolev nähtuma eelotsusetaotlus eelotsusetaotlus esitama kohus otsus tri üleandmine rumeenia ametiasutus õiguspärane seadus rahvusvaheline õigusabi kriminaalasi § 83 säte alune rakendama raamotsus 2002 / 584 artikkel 4 saksa õigus
2,C01PointnumeroteAltN,"29 Eelotsusetaotluse esitanud kohtu hinnangul on üleandmiseks vajalikud tingimused täidetud, kuna esiteks on see isik teadlikult hoidunud menetlustest – mille tulemusel on tehtud need Euroopa vahistamismäärused, mida see kohus peab täitma –, põgenedes Saksamaale ja takistades seega talle kohtukutse isiklikult kättetoimetamist, ning teiseks esindas seda isikut esimeses kohtuastmes tema valitud advokaat ja apellatsiooniastmes kohtu määratud kaitsja. TR väitis aga selles kohtus, et see üleandmine ei ole direktiivi 2016/343 artiklites 8 ja 9 sätestatud nõudeid arvestades õiguspärane, kuna Rumeenias ei ole mingit tagatist tema suhtes toimunud kriminaalmenetluse uuendamiseks.",NaN,C‑416/20 PPU,0,algus,29 eelotsusetaotluse esitanud kohtu hinnangul üleandmiseks vajalikud tingimused täidetud esiteks isik teadlikult hoidunud menetlustest – tulemusel euroopa vahistamismäärused kohus täitma – põgenedes saksamaale takistades kohtukutse isiklikult kättetoimetamist esindas isikut esimeses kohtuastmes valitud advokaat apellatsiooniastmes kohtu määratud kaitsja tr väitis kohtus üleandmine direktiivi 2016/343 artiklites 8 9 sätestatud nõudeid arvestades õiguspärane rumeenias tagatist toimunud kriminaalmenetluse uuendamiseks,29 eelotsusetaotlus esitama kohus hinnang üleandmine vajalik tingimus täidetud esiteks isik teadlikult hoiduma menetlus - tulemus euroopa vahistamismäärus kohus täitma - põgenema saksamaa takistama kohtukutse isiklikult kättetoimetamine esindama isik esimene kohtuaste valima advokaat apellatsiooniaste kohus määrama kaitsja tr väitma kohus üleandmine direktiiv 2016 / 343 artikk

In [9]:
data.shape

(14643, 8)

In [10]:
column_list=data['Lemma'].to_list()
column_list[0:100]

['27 väärima märkimine väljakujunenud kohtupraktika eltl artikkel 267 sätestama liikmesriik kohus euroopa kohus vaheline koostöömenetlus euroopa kohus ülesanne andma liikmesriik kohus tarvilik vastus võimaldama viimane pooleliolev kohtuasi lahendama arvestama euroopa kohus esitama küsimus vajadus sõnastama 2. aprill 2020. aasta kohtuotsus ruskama federacija c ‑ 897 / 19 ppu eu c : 2020 : 262 punkt 43 viidatud kohtupraktika',
 '28 käesolev nähtuma eelotsusetaotlus eelotsusetaotlus esitama kohus otsus tri üleandmine rumeenia ametiasutus õiguspärane seadus rahvusvaheline õigusabi kriminaalasi § 83 säte alune rakendama raamotsus 2002 / 584 artikkel 4 saksa õigus',
 '29 eelotsusetaotlus esitama kohus hinnang üleandmine vajalik tingimus täidetud esiteks isik teadlikult hoiduma menetlus - tulemus euroopa vahistamismäärus kohus täitma - põgenema saksamaa takistama kohtukutse isiklikult kättetoimetamine esindama isik esimene kohtuaste valima advokaat apellatsiooniaste kohus määrama kaitsja tr v

In [11]:
# koondab iga sõna vektorid üheks ning lisab need ühte listi
row_nr=[]
vektor=[]
i=0
for i in range(20):
    text = Text(column_list[i])
    text.analyse('segmentation')
    bert_tagger.tag(text)
    frame = text.bert_embeddings
    listike = np.sum(frame['bert_embedding'], axis=0)
    row_nr.append(i)
    vektor.append(listike[0])
    if i%1000==0:
      print(i)


0


In [14]:
np.save('embeddings.npy', vektor)

In [15]:
all_embeddings = np.load('embeddings.npy')

In [ ]:
vektor

[array([-10.64715969,  27.99872521, -13.71043664, ..., -30.27789933,
        -19.1062393 , -43.55224454]),
 array([-26.63466957,  27.78497875,  -5.99931939, ..., -25.21900728,
          0.75309072, -21.18545489]),
 array([-11.13486145,  66.16651857,  -2.58070103, ..., -40.24441222,
          5.83317178, -44.65653833]),
 array([-15.19114885,  88.73955798,  -4.15356229, ..., -30.92564445,
         19.16387664, -53.51790151]),
 array([ -46.14778613,   54.62818106,   -8.83204867, ...,  -16.54114341,
          57.68116563, -102.1500281 ]),
 array([  7.97319001,  61.13318682, -10.49001001, ..., -76.83479901,
        -26.90132058, -48.30903538]),
 array([-83.6508136 ,  31.42679492, -46.27356465, ..., -13.04678989,
         22.30989097, -96.39972558]),
 array([ -9.02303586,  16.88159287,  -5.17096528, ..., -31.11428437,
         14.72368754, -33.57538989]),
 array([-21.29316162,  51.72107974,  -9.62726994, ..., -44.72827189,
         15.66962427, -62.63337544]),
 array([-23.64502554,  22.78733

In [16]:
#testandmed
test_text = Text("käibemaksukohustuslaseks registreerimine tinkivinki")
test_text.analyse('segmentation')
bert_tagger.tag(test_text)
frame = test_text.bert_embeddings
proovivektoriks1=np.sum(frame["bert_embedding"], axis=0)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
def vordlus (vektor, row_nr, proovivektor, column_list):
    sarnased_maatriks=cosine_similarity(np.stack(vektor, axis=0), Y=proovivektor.reshape(1, -1), dense_output=True)
    sorteeritud_indeksid=np.argsort(np.squeeze(sarnased_maatriks))
    koige_sarnasem_idx=row_nr[sorteeritud_indeksid[len(sorteeritud_indeksid) - 1]]
    return column_list[koige_sarnasem_idx]

vordlus(vektor, row_nr, proovivektoriks1, column_list)

'28 käesolev nähtuma eelotsusetaotlus eelotsusetaotlus esitama kohus otsus tri üleandmine rumeenia ametiasutus õiguspärane seadus rahvusvaheline õigusabi kriminaalasi § 83 säte alune rakendama raamotsus 2002 / 584 artikkel 4 saksa õigus'